In [23]:
#https://medium.freecodecamp.org/how-to-build-a-twitter-sentiments-analyzer-in-python-using-textblob-948e1e8aae14
# colaborate with Pablo Mandiola

import os

import tweepy
import requests


from textblob import TextBlob

from IPython.core.display import clear_output
from time import time

import random
import shapely
from shapely.affinity import affine_transform
from shapely.geometry import shape,mapping,Point
import json

In [24]:
TWITTER_KEY = os.getenv('TWITTER_KEY')
TWITTER_SECRET = os.getenv('TWITTER_SECRET')
TWITTER_TOKEN = os.getenv('TWITTER_TOKEN')
TWITTER_TOKEN_SECRET = os.getenv('TWITTER_TOKEN_SECRET')

In [25]:
auth = tweepy.OAuthHandler(TWITTER_KEY, TWITTER_SECRET)
auth.set_access_token(TWITTER_TOKEN, TWITTER_TOKEN_SECRET)
api = tweepy.API(auth)

In [34]:
!ls docker

Dockerfile
nyc-borough-processed.geojson
nyc-zipcode-tabulation-areas.geojson
tweepy-streaming.py


In [35]:
import geopandas as gpd
geo = gpd.read_file("docker/nyc-borough-processed.geojson")
geo

,boro_code,boro_name,shape_area,shape_leng,geometry
0,1,Manhattan,636533240.297,359557.995733,(POLYGON ((-74.01092841268031 40.6844914725429...
1,2,Bronx,1186612476.84,462958.187796,(POLYGON ((-73.89680883223774 40.7958084451597...
2,5,Staten Island,1623920681.95,330432.867999,(POLYGON ((-74.05050806403247 40.5664220341608...
3,3,Brooklyn,1937566944.36,739945.437431,(POLYGON ((-73.86706149472118 40.5820879767934...
4,4,Queens,3044771532.61,895228.948392,(POLYGON ((-73.83668274106707 40.5949466970158...


In [18]:
# from pablo
#Based on https://codereview.stackexchange.com/questions/69833/generate-sample-coordinates-inside-a-polygon
# Pre-calculated triangulation to speed up process:
#Pre-calculated triangulation to speed-up process:
with open('docker/nyc-borough-processed.geojson',encoding="utf8") as json_file:  
    nyc_boroughs = json.load(json_file)
    
for boro in nyc_boroughs['features']:
    boro['geometry'] = shape(boro['geometry'])



JSONDecodeError: Expecting value: line 7 column 1 (char 6)

In [13]:
class MyStreamListener(tweepy.StreamListener):

    latestTweets = []
    totalTweets = 0
    nExactLocation = 0
    startTime = None
    
    def on_status(self, status):
        
        analysis = TextBlob(status.text)
        
        self.latestTweets.insert(0,(status.text, analysis.sentiment[0], status.coordinates))
        self.totalTweets += 1
        
        if (self.startTime is None):
            self.startTime = time()
        
        if (status.coordinates):
            self.nExactLocation += 1
        
        if len(self.latestTweets) > 10:
            self.latestTweets.pop()
        
        for tweet in self.latestTweets:
            print(tweet)
            
        print()
        elapsedTime = time() - self.startTime
        print('Tweets with lat/long: {} of {} ({:.2f}%); Frequency: {:.2f} tweets/s'.format(self.nExactLocation,
                                                                                              self.totalTweets,
                                                                                              self.nExactLocation/self.totalTweets*100,
                                                                                              self.totalTweets/elapsedTime))
        
        clear_output(wait=True)
        
    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_error disconnects the stream
            return False

In [14]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

In [15]:
#New York City bounding box according to Twitter Docs: -74,40,-73,41
#https://developer.twitter.com/en/docs/tweets/filter-realtime/guides/basic-stream-parameters
myStream.filter(locations=[-74.05,40.54,-73.7,40.92])

KeyboardInterrupt: 